# Dev: How to display thinking block

In [ ]:
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent)) 

In [ ]:
# What is 1+1?
assist_raw1 = """
<think>
Alright, the user just asked, "What is 1+1?" That seems pretty straightforward. They might be testing me or just starting with basic math. I should make sure to answer clearly and correctly. Maybe they're a student or someone looking for a quick calculation. I'll respond with the correct answer and offer further help in case they have more questions. Keeping it friendly and open-ended should encourage them to continue the conversation if they need more assistance.
</think>

1 + 1 equals **2**. How can I assist you further? 😊
"""

assist_raw1_partial = """
<think>
Alright, the user just asked, "What is 1+1?"
"""

## V1: `extract_from_think()` (fail just opening tag)

In [13]:
def extract_from_think(
    string: str,
):
    # Use regular expressions to find content between <think> tags
    import re
    match = re.search(rf"<think>\s*(.*?)\s*</think>", string, re.DOTALL)
    if not match:
        print(f"No <think> tags found in the input string.")
        return
    # Return the first match string
    return match.group(1)

In [15]:
extract_from_think(assist_raw1)

'Alright, the user just asked, "What is 1+1?" That seems pretty straightforward. They might be testing me or just starting with basic math. I should make sure to answer clearly and correctly. Maybe they\'re a student or someone looking for a quick calculation. I\'ll respond with the correct answer and offer further help in case they have more questions. Keeping it friendly and open-ended should encourage them to continue the conversation if they need more assistance.'

In [16]:
extract_from_think(assist_raw1_partial)

No <think> tags found in the input string.


## V2: `extract_thinking()`

In [ ]:
def extract_thinking(string: str):
    """
    Extract content from <think> tags, even if the closing tag isn't present yet (streaming case).
    
    Args:
        string: The input string that may contain <think> tags
        
    Returns:
        The content inside the <think> tags, or None if no tags are found
    """
    import re
    
    # Try to match complete <think> tags first
    complete_match = re.search(r"<think>\s*(.*?)\s*</think>", string, re.DOTALL)
    if complete_match:
        return complete_match.group(1)
    
    # If complete tags aren't found, check for partial (streaming) case
    last_think_pos = string.rfind("<think>")
    if last_think_pos != -1:
        # Extract everything after the last <think> tag (removing leading whitespace)
        return string[last_think_pos + len("<think>"):].lstrip()
    
    # No <think> tags found
    return ""

In [22]:
def display_thinking(string: str):
    thinking = extract_thinking(string)
    return f"""```
{thinking}
```"""
    

In [18]:
extract_thinking(assist_raw1)

'Alright, the user just asked, "What is 1+1?" That seems pretty straightforward. They might be testing me or just starting with basic math. I should make sure to answer clearly and correctly. Maybe they\'re a student or someone looking for a quick calculation. I\'ll respond with the correct answer and offer further help in case they have more questions. Keeping it friendly and open-ended should encourage them to continue the conversation if they need more assistance.'

In [19]:
extract_thinking(assist_raw1_partial)

'Alright, the user just asked, "What is 1+1?"\n'

In [23]:
display_thinking(assist_raw1)

'```\nAlright, the user just asked, "What is 1+1?" That seems pretty straightforward. They might be testing me or just starting with basic math. I should make sure to answer clearly and correctly. Maybe they\'re a student or someone looking for a quick calculation. I\'ll respond with the correct answer and offer further help in case they have more questions. Keeping it friendly and open-ended should encourage them to continue the conversation if they need more assistance.\n```'

## Extract Answer

In [24]:
def extract_answer(string: str):
    """
    Extract the answer part that appears after the closing </think> tag.
    
    Args:
        string: The input string that may contain <think> tags
        
    Returns:
        The content after the </think> tag, or an empty string if no closing tag is found
    """
    # Find the position of the last closing </think> tag
    last_closing_tag_pos = string.rfind("</think>")
    
    if last_closing_tag_pos != -1:
        # Extract everything after the closing tag (removing leading whitespace)
        answer = string[last_closing_tag_pos + len("</think>"):].lstrip()
        return answer
    
    # If no closing tag is found, it means we're still in the thinking phase
    # or there are no think tags at all
    return ""

In [25]:
extract_answer(assist_raw1)

'1 + 1 equals **2**. How can I assist you further? 😊\n'

In [26]:
extract_answer(assist_raw1_partial)

''